<div class="usecase-title">Urban Growth and Transit Planning for Melbourne</div>

<div class="usecase-authors"><b>Authored by:  </b> Manasa Nagaraja</div>

<div class="usecase-duration"><b>Duration:</b> {90} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Intermediate}</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<h2>Scenario</h2>
<div>The City of Melbourne is experiencing dynamic population changes across its various regions, presenting both opportunities and challenges for urban planning and public transportation. As the city's population density evolves, there is an urgent need to ensure that transportation services are adequately expanded and enhanced to meet the growing demand. This use case focuses on analyzing historical and projected population data to identify regions with significant population growth and to prioritize these areas for transportation infrastructure development.</div>

<h3>User Story</h3>

*  Transportation planner,wants to assess the current capacity and usage statistics of existing transportation modes, so that in this usecase we can propose where expansions or enhancements are needed based on population growth forecasts.

*   Urban & policy maker,wants to receive recommendations on areas for potential development  and to analyze historical population data across different regions within Melbourne, so that this usecase we can identify areas with significant population growth and prioritize them for transportation development and ensure these areas are with the necessary infrastructure to support population surges.



<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:


*   By using the "City of Melbourne Population Forecasts by Small Area 2021-2041" dataset to understand forecasted population growth in various regions, over different years, and the magnitude of growth in those areas.
*   Identify patterns and trends in population growth, highlighting regions that are projected to experience significant increases in population density.
*   Learn how to extract basic information from the datasets, understanding the significance of various features and data cleaning techniques to handle missing values and ensure data integrity.
*   Determine which features are essential for analysis and discard those that add unnecessary complexity or dimensionality.
*   Use geolocation data to visualize the exact locations of tram stops, bus stops, and metro stations, aiding in the identification of potential locations for future infrastructure.
*   Conduct initial visualizations to understand the overall distribution of features and identify any correlations among them
*   Implement machine learning models to predict the number of transportation services needed to accommodate population growth in specific locations
*   Use predictive analytics to provide actionable insights for urban planners and policy makers, helping them to plan for future transportation infrastructure.










<h3>Introduction</h3>

The aim of this use case is to conduct a comprehensive analysis of population growth trends across different regions within the City of Melbourne. As Melbourne's population continues to rise, understanding and planning for the future transportation needs of its residents,By examining historical and projected population data, we can identify regions that are expected to experience significant population surges.

By addressing these user stories, we aim to provide urban planners, policy makers, and transportation planners with the insights and data necessary to make informed decisions. This will ensure that Melbourne's transportation infrastructure keeps pace with its population growth, providing efficient and effective public mobility solutions for all residents.

<h4>Dataset List</h4>

*   City of Melbourne Population Forecasts by Small Area 2021-2041 — CoM Open Data Portal
*   City Circle tram stops — CoM Open Data Portal (melbourne.vic.gov.au)
*   Bus stops — CoM Open Data Portal (melbourne.vic.gov.au)
*   Metro Train Stations with Accessibility Information — CoM Open Data Portal (melbourne.vic.gov.au)








In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pandas as pd


In [2]:
# function to collect data
def collect_data(dataset_id):
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    dataset_id  = dataset_id
    format='csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit' : -1,
        'lang' : 'en',
        'timezone' : 'UTC'
    }

    response = requests.get(url,params=params)

    if response.status_code ==200:
        url_content = response.content.decode('utf-8')
        dataset = pd.read_csv(StringIO(url_content), delimiter=';')
        return dataset
    else:
        print(f"Request failed with status code: {response.status_code}")

In [4]:
# population dataset
dataset_id = 'city-of-melbourne-population-forecasts-by-small-area-2020-2040'
pop = collect_data(dataset_id)
print(len(pop))
pop.head()

17052


,geography,year,gender,age,value
0,City of Melbourne,2021,Female,Age 0-4,1943.0
1,City of Melbourne,2023,Female,Age 0-4,2477.0
2,City of Melbourne,2026,Female,Age 0-4,3595.0
3,City of Melbourne,2028,Female,Age 0-4,4083.0
4,City of Melbourne,2034,Female,Age 0-4,5242.0


In [5]:
# tram stop dataset
dataset_id = 'city-circle-tram-stops'
tram = collect_data(dataset_id)
print(len(tram))
tram.head(3)

28


,geo_point_2d,geo_shape,name,xorg,stop_no,mccid_str,xsource,xdate,mccid_int
0,"-37.81922319307822, 144.9614014008424","{""coordinates"": [144.9614014008424, -37.819223...",Market Street / Flinders Street,GIS Team,3,NaN,Mapbase,2011-10-18,3
1,"-37.821539117626855, 144.95356912978238","{""coordinates"": [144.95356912978238, -37.82153...",Victoria Police Centre / Flinders Street,GIS Team,D6,NaN,Mapbase,2011-10-18,6
2,"-37.815426586135686, 144.94512063442602","{""coordinates"": [144.94512063442602, -37.81542...",Central Pier / Harbour Esplanade,GIS Team,D2,NaN,Mapbase,2011-10-18,10


In [6]:
# bus stop dataset
dataset_id = 'bus-stops'
bus = collect_data(dataset_id)
print(len(bus))
bus.head(3)

309


,geo_point_2d,geo_shape,prop_id,addresspt1,addressp_1,asset_clas,asset_type,objectid,str_id,addresspt,asset_subt,model_desc,mcc_id,roadseg_id,descriptio,model_no
0,"-37.80384165792465, 144.93239283833262","{""coordinates"": [144.93239283833262, -37.80384...",0,76.819824,357,Signage,Sign - Public Transport,355,1235255,570648,NaN,Sign - Public Transport 1 Panel,1235255,21673,Sign - Public Transport 1 Panel Bus Stop Type 13,P.16
1,"-37.81548699581418, 144.9581794249902","{""coordinates"": [144.9581794249902, -37.815486...",0,21.561304,83,Signage,Sign - Public Transport,600,1231226,548056,NaN,Sign - Public Transport 1 Panel,1231226,20184,Sign - Public Transport 1 Panel Bus Stop Type 8,P.16
2,"-37.81353897396532, 144.95728334230756","{""coordinates"": [144.95728334230756, -37.81353...",0,42.177187,207,Signage,Sign - Public Transport,640,1237092,543382,NaN,Sign - Public Transport 1 Panel,1237092,20186,Sign - Public Transport 1 Panel Bus Stop Type 8,P.16


In [7]:
# metro stations dataset
dataset_id = 'metro-train-stations-with-accessibility-information'
metro = collect_data(dataset_id)
print(len(metro))
metro.head(3)

219


,geo_point_2d,geo_shape,he_loop,lift,pids,station
0,"-37.77839599999999, 145.031251","{""coordinates"": [145.031251, -37.7783959999999...",No,No,Dot Matrix,Alphington
1,"-37.86724899999996, 144.830604","{""coordinates"": [144.830604, -37.8672489999999...",No,No,LCD,Altona
2,"-37.761897999999974, 144.96056099999998","{""coordinates"": [144.96056099999998, -37.76189...",No,No,No,Anstey
